In [1]:
exec(open("./funcs/tool_funcs.py").read())


In [2]:
# For the widest path problem, we need a Max-Heap, rather than a Min-Heap. 
# We use some techniques to convert a Min-Heap into a Max-Heap.


def dijkstra_show_path_widest_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [0] * n
    previous_nodes = [-1] * n
    distances[start] = np.inf
    
    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
    priority_queue = [(-np.inf, start)]  # (distance, node)

 
    
    while priority_queue:
       
        
        # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
        current_distance, current_node = heapq.heappop(priority_queue)
        current_distance = -current_distance

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance < distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight < np.inf:  
                distance = min(current_distance, weight)

                # Update distance if it's better
                if distance > distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    
                    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
                    heapq.heappush(priority_queue, (-distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [3]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [4]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [5]:
def cal_candidate_node_list(i, j, widest_path_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if widest_path_matrix[i,j] > min(widest_path_matrix[i,t], widest_path_matrix[t,j]):
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [6]:

def cal_path_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list(i, j, widest_path_matrix, X_distance_matrix)
    # print(candidate_node_list)
   
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path_widest_path(small_matrix, 0, len(candidate_node_list) - 1)
    path = translate_path(path, candidate_node_list)
    
    return path


In [7]:
N = 100

X_distance_matrix = create_distance_matrix(N)

In [8]:
# X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")


In [9]:

# For the widest path problem, a node's bandwidth to itself should be infinity.


N = len(X_distance_matrix)
for i in range(N):
    X_distance_matrix[i,i] = np.inf


In [10]:


 
widest_path_matrix = floyd_warshall_widest_path(X_distance_matrix)
 

In [11]:
i = 10
j = 50

path1 = dijkstra_show_path_widest_path(X_distance_matrix, i, j)  
path2 = cal_path_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix) 

print(path1)
print(path2)


    

[10, 65, 92, 90, 77, 58, 78, 72, 26, 85, 94, 6, 97, 20, 50]
[10, 65, 92, 90, 77, 58, 78, 72, 26, 85, 94, 6, 97, 20, 50]


In [12]:
start = time.time()
for i in range(N):
    for j in range(N):
#         if i != j:
        if i == 2:
            path2 = cal_path_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix)
end = time.time()

 
time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

print("time_used1_warm: ", time_used1_warm)            


time_used1_warm:  0.303


In [13]:
start = time.time()
for i in range(N):
    for j in range(N):
#         if i != j:
        if i == 2:
            path1 = dijkstra_show_path_widest_path(X_distance_matrix, i, j)
end = time.time()

 
time_used2_cold = end - start
time_used2_cold = np.round(time_used2_cold, 3)

print("time_used2_cold: ", time_used2_cold) 


time_used2_cold:  0.179


In [14]:
print("Ratio: ", np.round(time_used1_warm/time_used2_cold, 3))


Ratio:  1.693


In [15]:
# It seems a warm-start calculation of widest path can be less efficient than cold-start. 
# The speculative reason is that the size of candidate_node_list is quite large.
